In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split


from tensorflow.keras.preprocessing.image import ImageDataGenerator

import cv2


from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten

from tensorflow.keras.models import model_from_json

In [2]:
print(os.listdir('../own_dataset/diferent_dice/train'))

['red_1d6', 'red_1d10', 'blue_1d20', 'blue_1d6']


In [3]:
train_path= ('../own_dataset/diferent_dice/train')
test_path= ('../own_dataset/diferent_dice/test')

In [4]:
batch_size_train=30  #,batch_size= batch_size_train
batch_size_valid=30
targetsize= 50
datagen=ImageDataGenerator(rotation_range=180,height_shift_range=0.1,rescale=1.0/255.0
                           ,zoom_range=[0.7,1.1],brightness_range=[0.4,1.2])
train_batches= datagen.flow_from_directory(train_path, target_size=(targetsize,targetsize), 
                              classes=os.listdir('../own_dataset/diferent_dice/train'),
                              batch_size= batch_size_train)
test_batches= datagen.flow_from_directory(test_path,  target_size=(targetsize,targetsize), 
                              classes=os.listdir('../own_dataset/diferent_dice/train'),
                              batch_size= batch_size_train)

Found 10186 images belonging to 4 classes.
Found 2002 images belonging to 4 classes.


In [5]:
def plots(ims, figsize=(20,10), rows=1, interp= False, titles= None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims= ims.transpose((0,1,2,3))
    f= plt.figure(figsize=figsize)
    cols= len(ims)//rows if len(ims) %2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=12)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

In [6]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 *0.95**x)

In [7]:
"""

nfilters = [8, 16]
kernel_sizes = [(4,4), (2,2)]


model = Sequential()
# CONV1 (ReLU) > POOL1
model.add(Conv2D(nfilters[0], kernel_sizes[0], # 8 filters, 4x4
                 strides=(1,1),
                 padding='same', 
                 input_shape=(targetsize,targetsize, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(8,8),
                       strides=(8,8),
                       padding='same'))
# CONV2 (ReLU) > POOL2
model.add(Conv2D(nfilters[1], kernel_sizes[1], # 16 filters, 2x2
                 strides=(1,1),
                 padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(4,4),
                       strides=(4,4),
                       padding='same'))

# Fully connected layer with softmax
model.add(Flatten())
model.add(Dense(6, activation='softmax'))
model.summary()
"""

"\n\nnfilters = [8, 16]\nkernel_sizes = [(4,4), (2,2)]\n\n\nmodel = Sequential()\n# CONV1 (ReLU) > POOL1\nmodel.add(Conv2D(nfilters[0], kernel_sizes[0], # 8 filters, 4x4\n                 strides=(1,1),\n                 padding='same', \n                 input_shape=(targetsize,targetsize, 3),activation='relu'))\nmodel.add(MaxPooling2D(pool_size=(8,8),\n                       strides=(8,8),\n                       padding='same'))\n# CONV2 (ReLU) > POOL2\nmodel.add(Conv2D(nfilters[1], kernel_sizes[1], # 16 filters, 2x2\n                 strides=(1,1),\n                 padding='same',activation='relu'))\nmodel.add(MaxPooling2D(pool_size=(4,4),\n                       strides=(4,4),\n                       padding='same'))\n\n# Fully connected layer with softmax\nmodel.add(Flatten())\nmodel.add(Dense(6, activation='softmax'))\nmodel.summary()\n"

In [8]:

"""
model.compile(Adam(lr=.0002), loss='categorical_crossentropy', metrics= ['accuracy'])
history= model.fit_generator(train_batches, steps_per_epoch= len(train_batches) , callbacks=[reduce_lr],
                             validation_data=train_batches, validation_steps= len(train_batches), 
                             epochs=500)
"""

"\nmodel.compile(Adam(lr=.0002), loss='categorical_crossentropy', metrics= ['accuracy'])\nhistory= model.fit_generator(train_batches, steps_per_epoch= len(train_batches) , callbacks=[reduce_lr],\n                             validation_data=train_batches, validation_steps= len(train_batches), \n                             epochs=500)\n"

In [9]:
#cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=os.getcwd(),save_weights_only=True,verbose=1)

In [10]:
#model.save('/home/ordovas/IRONHACK/dice-scores-recognition/model_d6_augmented_c1.h5')

In [11]:

model2 = Sequential()
model2.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(targetsize,targetsize, 3)) )
model2.add(MaxPooling2D((2, 2)))

model2.add(Conv2D(filters=64, kernel_size=(3,3), padding='SAME', activation='relu'))
model2.add(MaxPooling2D((2, 2)))


model2.add(Conv2D(filters=128, kernel_size=(3,3), padding='SAME', activation='relu'))
model2.add(MaxPooling2D((2, 2)))

#model2.add(Conv2D(filters=256, kernel_size=(3,3), padding='SAME', activation='relu'))
#model2.add(MaxPooling2D((2, 2)))

#model2.add(Conv2D(filters=512, kernel_size=(3,3), padding='SAME', activation='relu'))
#model2.add(MaxPooling2D((2, 2)))

model2.add(Flatten())
model2.add(Dense(len(os.listdir('../own_dataset/diferent_dice/train')), activation='softmax'))
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 50, 50, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 25, 25, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0

In [12]:
model2.compile(Adam(lr=.0002), loss='categorical_crossentropy', metrics= ['accuracy'])
history2= model2.fit_generator(train_batches, steps_per_epoch= len(train_batches) , 
                             validation_data=test_batches, validation_steps= len(test_batches), 
                             epochs=10)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
340/340 [==============================] - 601s 2s/step - loss: 0.2629 - accuracy: 0.9169 - val_loss: 0.0721 - val_accuracy: 0.9775
Epoch 2/10
340/340 [==============================] - 276s 813ms/step - loss: 0.0364 - accuracy: 0.9930 - val_loss: 0.0475 - val_accuracy: 0.9905
Epoch 3/10
340/340 [==============================] - 143s 422ms/step - loss: 0.0289 - accuracy: 0.9937 - val_loss: 0.0383 - val_accuracy: 0.9890
Epoch 4/10
340/340 [==============================] - 150s 442ms/step - loss: 0.0254 - accuracy: 0.9946 - val_loss: 0.0567 - val_accuracy: 0.9845
Epoch 5/10
340/340 [==============================] - 145s 427ms/step - loss: 0.0230 - accuracy: 0.9953 - val_loss: 0.0315 - val_accuracy: 0.9940
Epoch 6/10
340/340 [==============================] - 168s 495ms/step - loss: 0.0231 - accuracy: 0.9956 - val_loss: 0.0512 - val_accuracy: 0.9880
Epoch 7/10
340/340 [==============================]

In [13]:
history2= model2.fit_generator(train_batches, steps_per_epoch= len(train_batches) , 
                             validation_data=test_batches, validation_steps= len(test_batches), 
                             epochs=3)

Epoch 1/3
340/340 [==============================] - 105s 310ms/step - loss: 0.0127 - accuracy: 0.9973 - val_loss: 0.1315 - val_accuracy: 0.9630
Epoch 2/3
340/340 [==============================] - 106s 312ms/step - loss: 0.0129 - accuracy: 0.9964 - val_loss: 0.0406 - val_accuracy: 0.9920
Epoch 3/3
340/340 [==============================] - 109s 320ms/step - loss: 0.0147 - accuracy: 0.9969 - val_loss: 0.0205 - val_accuracy: 0.9960


In [14]:
cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=os.getcwd(),save_weights_only=True,verbose=1)

In [15]:
model2.save('model_4difdices_final.h5')

In [16]:
model2_json = model2.to_json()
with open("model_4difdices_final.json", "w") as json_file:
    json_file.write(model2_json)

In [17]:
# https://machinelearningmastery.com/save-load-keras-deep-learning-models/

